<a href="https://colab.research.google.com/github/jamg-upv/LLMforSLRscreening/blob/main/UBdocSchoolMAy25/chatgptAPI_PROMPT_UBschoolMay25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Descripción del caso de uso GPTUS01-entity_extraction
basado en Reto21-chatgptAPI-CasoUso1HIWPclasificacion.ipynb

Clonar el bot que tengo en poe.com sobre entity extraction y clasificación de resumenes de artículos.

LA entrada es una tabla con dos columnas
ID|abstract
El resultado quiero que me genere una tabla con
Id|Entity extraction separada por ";"|clasificacion|rationale para la clasificación

Manualmente copiando y pegando el input y los resultados (de 20 en 20) me funciona dentro de la ventana de contexto. Pero quiero evitar tener que hacerlo a mano.

# Archivo de trabajo

In [ ]:
# En Python, puedes usar comillas simples o dobles para definir una cadena de texto (string). Ambas son válidas y no hay diferencia funcional entre ellas.
#  si tu cadena de texto contiene una comilla simple (como en "I'm"), puedes usar comillas dobles para evitar tener que escapar la comilla simple. De manera similar, si tu cadena de texto contiene una comilla doble, puedes usar comillas

# RutaData = input("Ruta completa al archivo (ejemplo: https://gdu.uv.es/sh/alfresco.xls ): ")

RutaData="savedrecsSET1311results.xlsx"

# Este código lee el archivo XLS ubicado en la URL especificada y lo carga en un DataFrame de pandas llamado DataPapers.
# Debes tener en cuenta que read_excel() necesita la biblioteca openpyxl o xlrd para leer archivos de Excel,
# si no la tienes instalada, puedes hacerlo con pip install openpyxl o pip install xlrd.
# Además, también necesitas tener la biblioteca requests para leer archivos desde una URL. Si no la tienes instalada, puedes hacerlo con pip install requests.

import pandas as pd

# Leemos el archivo en un DataFrame
DataPapers = pd.read_excel(RutaData)

DataPapers['Prompt'] = DataPapers['Id'].astype(str) + "# "  + DataPapers['Title'] + ". "+ DataPapers['Abstract']
DataPapers

# Definimos las columnas que queremos mantener
columnas = ["Id", "Title","Abstract", "Prompt"]

# Seleccionamos las columnas y las primeras 128 filas
# DataPapers = DataPapers[columnas].iloc[:10]

# Seleccionamos las columnas y TODAS las filas (no solo las primeras 10)
DataPapers = DataPapers[columnas]  # Eliminamos el .iloc[:10]


DataPapers



,Id,Title,Abstract,Prompt
0,id0001,Study on Data Analysis of Assessment in Class ...,"Nowadays, students' evaluation of teaching is ...",id0001# Study on Data Analysis of Assessment i...
1,id0002,Development and Piloting of Online Student Eva...,This paper presents the experience of Makerere...,id0002# Development and Piloting of Online Stu...
2,id0004,On Student Evaluation of Teaching and Improvem...,Student evaluation of teaching is a fundamenta...,id0004# On Student Evaluation of Teaching and ...
3,id0006,The student evaluation of teaching and likabil...,For several decades research into the student ...,id0006# The student evaluation of teaching and...
4,id0008,A literature review of the research on student...,Students' evaluation of teaching is a teaching...,id0008# A literature review of the research on...
...,...,...,...,...
953,id1304,Evaluating college students' evaluations of a ...,Aims: Do college students' ratings of a profes...,id1304# Evaluating college students' evaluatio...
954,id1305,Is lecture dead? A preliminary study of medica...,Objectives: To investigate medical students' p...,id1305# Is lecture dead? A preliminary study o...
955,id1307,Shifting from face-to-face to online teaching ...,"As a result of the COVID-19 pandemic, many fac...",id1307# Shifting from face-to-face to online t...
956,id1310,Student Feedback & Systematic Evaluation of Te...,"Rapid advances in technology, economic globali...",id1310# Student Feedback & Systematic Evaluati...


#introducir api key OpenAI (jecutar una vez cada sesión)

In [ ]:
#manual
import getpass

# Solicitar la entrada al usuario
keyOpenAi= getpass.getpass("Introduce API KEY sin comillas: ")

Introduce API KEY sin comilles: ··········


In [ ]:
#automatico

from google.colab import userdata
keyOpenAi = userdata.get('KeyOpenAi')

In [ ]:
!pip install openai

import os
from openai import OpenAI

# Inicialización del cliente con la API moderna
client = OpenAI(api_key=keyOpenAi)

# Selección del modelo precio  Input	Cached input	Output  https://platform.openai.com/docs/pricing
# ModelUsed="gpt-4.1-2025-04-14"        #$2.00 $0.50  S8.00
ModelUsed="gpt-4o-mini-2024-07-18"  #$0.15 $0.075 $0.60
# ModelUsed="o4-mini-2025-04-16"      #$1.10 $0.275 $4.40



# Convertir en un bucle y usar funcion para tener un codigo más limpio

In [ ]:
PromtSystemB = '''
# Role and Context
You are an [Pedagogy] scientist specializing in screening scientific articles related to [Student evaluation of teaching]. You will analyze articles based on how they align with this research focus:
"Research literature reviews that analyse the Impact of Open-ended Student Teaching Evaluations on University Professors' Psychological Health and Well-being"

# Key Definition and Components
Student Evaluation of Teaching (SET) refers to the systematic process of collecting feedback from students about their educational experiences, instructor effectiveness, and course quality. These evaluations typically occur at the end of a course or academic term and involve students rating various aspects of teaching performance and course design through questionnaires or surveys. SETs are used for various purposes including:

Synonyms and Related Terms
* Course evaluations
* Teaching evaluations
* Student ratings of instruction (SRI)
* Student feedback questionnaires
* Student assessment of teaching (SAT)
* Student perception of teaching (SPOT)
* Student evaluations of educational quality (SEEQ)
* Student ratings of teaching effectiveness (SRTE)
* Student course experience questionnaires (SCEQ)
* Teaching effectiveness measures


# Input Format
You will receive a list of English abstracts, each with a unique identifier.

# Processing Instructions
For each abstract, follow these four steps:

1. Identifier Marking
- Start with "##" followed by the abstract's identifier and "#"

2. Entity Extraction
- Extract relevant entities from the abstract
- Separate entities with semicolons (;)
- Focus on extracting:
  * Evaluation format (open or close ended questions)
  * Performance measures
  * Key variables
- End with "#"

3. Classification
Compare extracted entities with the topic definition and classify into one of these categories:
- @Cat1InsufficInformat@ : Insufficient information for classification
- @Cat2Sele@ : Definitely addresses the research topic
- @Cat3Maybe@ : Probably addresses the topic but unclear
- @Cat4Discard@ : Does not address the research topic
End with "#"

 4. Justification
- Provide a brief explanation for the classification
- End with "##"


# Classification Criteria

To be classified as @Cat2Sele@, abstract must show:
1. Clear focus on studetn evaluation survey (per definition)
2. Open Ended questions

# Sample Analysis
Input format:
ID: [ABC123]   Abstract: [Abstract text]

Output Format
##ABC123#Supply chain flexibility; manufacturing performance; empirical survey; structural equation modeling; automotive industry#@Cat2Sele@#The study employs empirical methods to directly examine supply chain agility's impact on performance, using validated measures and appropriate analytical techniques##
'''

# #  la "temperatura" es un hiperparámetro que controla la aleatoriedad de las predicciones del modelo.
#  Un valor de temperatura más alto (como 0.8 o 1.0) hará que el modelo haga predicciones más diversas y creativas,
#  mientras que un valor más bajo (como 0.2 o 0.3) hará que las predicciones sean más concentradas y deterministas.
# # Dado que tu tarea implica la extracción de entidades, la clasificación y la explicación,
# querrías que las respuestas del modelo sean coherentes y directas en lugar de ser demasiado creativas.
# Por lo tanto, te recomendaría una temperatura más baja, posiblemente alrededor de 0.3.

Temperatura=0.5 #por debajo de 0.5 pocas veces sigue las instrucciones (no llega al cuarto paso o no pone los # donde toca)



In [ ]:
# Crear un DataFrame vacío
df3 = pd.DataFrame(columns=['GPTId', 'GPTContent','GPTResponse','GPTUsage','PromtAgregado'])

In [ ]:
# código refacturizado: para hacer esto
#  extrayendo el texto de una fila de un dataframe,
#   usándolo como entrada para la API de ChatCompletion de OpenAI,
#   e insertando la respuesta en otro dataframe.
# Este código asume que tienes una cierta flexibilidad en la elección de tus filas de inicio y fin.
#  Si siempre estás procesando la siguiente fila y no tienes un rango fijo,
#   podrías considerar almacenar la "fila actual" en una variable persistente o en un archivo, y actualizarla cada vez que ejecutes la función.

# La función generate_response_and_update_df toma el DataFrame donde quieres guardar la respuesta,
#  el índice de la fila a actualizar, el modelo de OpenAI a utilizar, los mensajes del sistema y del usuario, y la temperatura para la generación de texto.
#  Luego realiza la llamada a la API de OpenAI, imprime la información de uso y el contenido de la respuesta, actualiza el DataFrame con la información de la respuesta y finalmente devuelve el DataFrame actualizado.
import time

def generate_response_and_update_df(df, row, model, system_prompt, user_prompt, temperature):
    # Llamada a la API de OpenAI con la sintaxis actualizada
    response = client.chat.completions.create(
      model=model,
      messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
      temperature=temperature
    )

    # Imprimir la información de los tokens y el contenido de respuesta
    print(response.usage)
    print(response.choices[0].message.content)

    # Añadir la respuesta GPT al DataFrame
    df.loc[row, 'GPTId'] = row
    df.loc[row, 'GPTContent'] = response.choices[0].message.content
    df.loc[row, 'GPTResponse'] = str(response)
    df.loc[row, 'GPTUsage'] = str(response.usage)
    df.loc[row, 'PromtAgregado'] = user_prompt

    return df

# Llamada a la función en un bucle
start_row = 0
end_row = 500
fixed_delay = 0.05  # la idea es no pasarse de 200 peticiones por minuto, incluso sin delay va a unas 30 peticiones por minuto

for row in range(start_row, end_row):
    PromtUser1 = DataPapers['Prompt'].iloc[row]
    df3 = generate_response_and_update_df(df3, row, ModelUsed, PromtSystemB, PromtUser1, Temperatura)
    print(f"Procesada fila {row + 1} de {end_row}")

    # Guardar periódicamente para no perder progreso
    if row % 50 == 0 and row > 0:  # Cada 50 filas
        df3.to_csv(f"resultados_parciales_hasta_fila_{row}.csv", index=False)
        print(f"Progreso guardado hasta la fila {row}")

    # Pausa fija entre peticiones
    if row < end_row - 1:  # No esperar después de la última fila
        time.sleep(fixed_delay)

# Guardar resultado final
df3.to_csv("resultados_finales.csv", index=False)
print("Proceso completado!")

df3





In [ ]:
# Guardamos el DataFrame en un archivo Excel

# Definimos la ruta y el nombre del archivo incluyendo la fecha actual
from datetime import datetime
fecha_actual = datetime.now().strftime("%Y-%m-%d %H:%M")
nombre_base = f"{ModelUsed} B2-{Temperatura}"
nombre_fichero = f"{nombre_base}_{fecha_actual}.xlsx"

df3.to_excel(nombre_fichero, index=False)

# mejoras

